In [6]:
!pip install geohash2

  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15544 sha256=d36f799fae981afe0e5be2c05195a45c3b711486278baaaee7c10bb792c2fca6
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\2d\0c\66\dcd768c9e7c26fc81ec59bfe24af9d780fc9dbdf7f90b0cf00
Successfully built geohash2


In [7]:
import requests 
import pandas as pd
import geohash2

#  직방 원룸매물정보수집
  -절차 
   동이름  >  위도 경도 수집 1

  위도경도 > geohash 변환

  geohash > 매물아이디  2 
  매물아이디 >  매물 정보 3
requests 를 3번해야함


In [56]:
#  동이름 - > 위도 경도 
addr='강남구 역삼동'
url= f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
response = requests.get(url)
data=response.json()['items'][0]
lat, lng = data['lat'], data['lng']
lat , lng


(37.495365142822266, 127.03306579589844)

In [19]:
#  위도 경도  - > geohash 
#  precision 이커질수록 영역이 작이짐
geohash =geohash2.encode(lat, lng , precision = 5)
geohash

'wydm6'

post할때 어떤 파라미터가 필요한지는 개발자 도구의 Payload에서 찾아보기

In [29]:
#  geohash  - > 매물아이다 
url =f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
response =requests.get(url)
items=response.json()['items']
ids = [item['item_id'] for item in items]

In [31]:
ids[:5]

[32819635, 32838798, 32872608, 32838700, 32767020]

In [33]:
#  매물아이디  - >  매물 정보
url = 'https://apis.zigbang.com/v2/items/list'
params= {
    'domain': "zigbang",
    'withCoalition': 'true',
    'item_ids' : ids[:900]
}

response =requests.post(url,params)
response

<Response [200]>

In [ ]:
items=response.json()['items']
items[:2]
#  리스트안에 딕셔너리면 바로 DF 하면됨

In [44]:
#  maxrow max columns 설정하기 
pd.options.display.max_columns = 30

In [42]:
items = response.json()["items"] 
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
df = pd.DataFrame(items)[columns] 
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
898,32809154,월세,120,70,19.83,서울시 강남구 역삼동,9
899,32844154,월세,100,100,29.75,서울시 강남구 역삼동,9


In [61]:
def oneroom(addr):
    url= f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'   #매물데이터 크롤링하기
    response = requests.get(url)
    data=response.json()['items'][0]
    lat, lng = data['lat'], data['lng'] 
    
    geohash =geohash2.encode(lat, lng , precision = 5)    
    #위도경도 해쉬넘버로 인코딩 
    
    url =f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response =requests.get(url)
    items=response.json()['items']
    ids = [item['item_id'] for item in items]
    
    url = 'https://apis.zigbang.com/v2/items/list'
    params= {
    'domain': "zigbang",
    'withCoalition': 'true',
    'item_ids' : ids[:900]
        }   
    response =requests.post(url,params)
    
    items = response.json()["items"]
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    return pd.DataFrame(items)[columns] 

In [62]:
addr = '마포구 합정동'
df2 = oneroom(addr)

In [63]:
df2

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32864713,전세,9000,0,21.49,서울시 영등포구 당산동4가,13
1,32694522,전세,9000,0,19.83,서울시 영등포구 당산동4가,13
2,32822043,전세,9000,0,16.53,서울시 영등포구 당산동4가,13
3,32827028,전세,9000,0,19.83,서울시 영등포구 당산동4가,13
4,32856780,전세,7000,0,19.83,서울시 영등포구 당산동4가,13
...,...,...,...,...,...,...,...
741,32831032,월세,200,65,18.18,서울시 마포구 성산동,7
742,32775788,전세,16500,0,19.83,서울시 마포구 성산동,1
743,32589981,전세,43000,0,49.00,서울시 마포구 중동,7
744,32862490,전세,43000,0,31.74,서울시 마포구 중동,5


In [67]:
%%writefile zigbang.py
import requests 
import pandas as pd
import geohash2


def oneroom(addr):
    url= f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'   #매물데이터 크롤링하기
    response = requests.get(url)
    data=response.json()['items'][0]
    lat, lng = data['lat'], data['lng'] 
    
    geohash =geohash2.encode(lat, lng , precision = 5)    
    #위도경도 해쉬넘버로 인코딩 
    
    url =f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response =requests.get(url)
    items=response.json()['items']
    ids = [item['item_id'] for item in items]
    
    url = 'https://apis.zigbang.com/v2/items/list'
    params= {
    'domain': "zigbang",
    'withCoalition': 'true',
    'item_ids' : ids[:900]
        }   
    response =requests.post(url,params)
    
    items = response.json()["items"]
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    return pd.DataFrame(items)[columns] 
#  모듈 파일만들기 .py

Overwriting zigbang.py
